In [293]:
import pandas as pd
import optuna as opt
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.impute import KNNImputer

def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 750),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'depth': trial.suggest_int('depth', 2, 10),
    }

    model = CatBoostClassifier(**params, random_state=42)

    score = cross_val_score(model, X, y, cv=3, scoring='accuracy').mean()

    return score

df = pd.read_csv('../train.csv')
df = df.drop(columns=['Name', 'PassengerId'])

binary_features = ['VIP', 'CryoSleep']
cryo_sleep_depending_features = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
categorical_features_without_cabin = ['HomePlanet', 'Destination']
categorical_features_cleared = ['HomePlanet', 'Destination', 'Deck', 'Side']
numerical_features = ['ShoppingMall', 'Spa', 'VRDeck', 'RoomService', 'FoodCourt', 'Age']
imputer = KNNImputer(n_neighbors=1)

for i in range(len(df)):
    if df.at[i, 'CryoSleep'] == True:
        for feature in cryo_sleep_depending_features:
            df.at[i, feature] = 0.0

rows_with_many_missing_values = df[df.isna().sum(axis=1) > 1]
rows_with_one_missing_value = df[df.isna().sum(axis=1) == 1]
df = df.dropna()     
df[["Deck", "CabinNumber", "Side"]] = df["Cabin"].str.split("/", expand=True)
df = df.drop(columns=['Cabin', 'CabinNumber'])
df = df.astype({'CryoSleep': int, 'VIP': int, 'Transported': int})
df = pd.get_dummies(df, columns=categorical_features_cleared)

for feature in numerical_features:
    rows_with_missing_feature = rows_with_one_missing_value[rows_with_one_missing_value[feature].isna()] 
    rows_with_missing_feature[["Deck", "CabinNumber", "Side"]] = rows_with_missing_feature["Cabin"].str.split("/", expand=True)
    rows_with_missing_feature = rows_with_missing_feature.drop(columns=['Cabin', 'CabinNumber'])
    rows_with_missing_feature = rows_with_missing_feature.astype({'CryoSleep': int, 'VIP': int, 'Transported': int})
    rows_with_missing_feature = pd.get_dummies(rows_with_missing_feature, columns=categorical_features_cleared)
    df = pd.concat([df, rows_with_missing_feature], ignore_index=True)
    df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

for feature in categorical_features_without_cabin:
    rows_with_missing_feature = rows_with_one_missing_value[rows_with_one_missing_value[feature].isna()] 
    rows_with_missing_feature[["Deck", "CabinNumber", "Side"]] = rows_with_missing_feature["Cabin"].str.split("/", expand=True)
    rows_with_missing_feature = rows_with_missing_feature.drop(columns=['Cabin', 'CabinNumber'])
    rows_with_missing_feature = rows_with_missing_feature.astype({'CryoSleep': int, 'VIP': int, 'Transported': int})
    rows_with_missing_feature = pd.get_dummies(rows_with_missing_feature, columns=categorical_features_cleared)
    df = pd.concat([df, rows_with_missing_feature], ignore_index=True)
    df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

for feature in binary_features:
    rows_with_missing_feature = rows_with_one_missing_value[rows_with_one_missing_value[feature].isna()] 
    rows_with_missing_feature[["Deck", "CabinNumber", "Side"]] = rows_with_missing_feature["Cabin"].str.split("/", expand=True)
    rows_with_missing_feature = rows_with_missing_feature.drop(columns=['Cabin', 'CabinNumber'])
    rows_with_missing_feature = pd.get_dummies(rows_with_missing_feature, columns=categorical_features_cleared)
    df = pd.concat([df, rows_with_missing_feature], ignore_index=True)
    df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

rows_with_missing_feature = rows_with_one_missing_value[rows_with_one_missing_value['Cabin'].isna()] 
rows_with_missing_feature = rows_with_missing_feature.drop(columns='Cabin')
rows_with_missing_feature = rows_with_missing_feature.astype({'CryoSleep': int, 'VIP': int, 'Transported': int})
rows_with_missing_feature = pd.get_dummies(rows_with_missing_feature, columns=categorical_features_without_cabin)
df = pd.concat([df, rows_with_missing_feature], ignore_index=True)
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

rows_with_many_missing_values = rows_with_many_missing_values.drop(columns='Cabin')

df = pd.concat([df, rows_with_missing_feature], ignore_index=True)
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

target = 'Transported'
X = df.drop(columns=target)
y = df[target]

study = opt.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

best_params = study.best_params
print("Лучшие параметры:", best_params)

C:\Users\vitya\AppData\Local\Temp\ipykernel_5568\3372871564.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows_with_missing_feature[["Deck", "CabinNumber", "Side"]] = rows_with_missing_feature["Cabin"].str.split("/", expand=True)
C:\Users\vitya\AppData\Local\Temp\ipykernel_5568\3372871564.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows_with_missing_feature[["Deck", "CabinNumber", "Side"]] = rows_with_missing_feature["Cabin"].str.split("/", expand=True)
C:\Users\vitya\AppData\Local\Temp\ipyk

0:	learn: 0.6926290	total: 2.43ms	remaining: 1.28s
1:	learn: 0.6920721	total: 4.53ms	remaining: 1.19s
2:	learn: 0.6916042	total: 6.68ms	remaining: 1.17s
3:	learn: 0.6910885	total: 8.86ms	remaining: 1.16s
4:	learn: 0.6905400	total: 11ms	remaining: 1.15s
5:	learn: 0.6899923	total: 13ms	remaining: 1.13s
6:	learn: 0.6894523	total: 15.2ms	remaining: 1.13s
7:	learn: 0.6889856	total: 17ms	remaining: 1.11s
8:	learn: 0.6884878	total: 19.1ms	remaining: 1.1s
9:	learn: 0.6880169	total: 21ms	remaining: 1.09s
10:	learn: 0.6874607	total: 22.9ms	remaining: 1.08s
11:	learn: 0.6870086	total: 24.8ms	remaining: 1.07s
12:	learn: 0.6864605	total: 26.6ms	remaining: 1.05s
13:	learn: 0.6860385	total: 28.4ms	remaining: 1.04s
14:	learn: 0.6855559	total: 30.2ms	remaining: 1.03s
15:	learn: 0.6850781	total: 32.2ms	remaining: 1.03s
16:	learn: 0.6846141	total: 34ms	remaining: 1.02s
17:	learn: 0.6841060	total: 35.7ms	remaining: 1.01s
18:	learn: 0.6835585	total: 37.5ms	remaining: 1.01s
19:	learn: 0.6830730	total: 39.3m

[I 2024-02-17 09:32:57,389] Trial 0 finished with value: 0.7781597341583592 and parameters: {'iterations': 529, 'learning_rate': 0.0010774297939288773, 'depth': 5}. Best is trial 0 with value: 0.7781597341583592.


520:	learn: 0.5414370	total: 1s	remaining: 15.4ms
521:	learn: 0.5412968	total: 1.01s	remaining: 13.5ms
522:	learn: 0.5411015	total: 1.01s	remaining: 11.6ms
523:	learn: 0.5409586	total: 1.01s	remaining: 9.64ms
524:	learn: 0.5408172	total: 1.01s	remaining: 7.72ms
525:	learn: 0.5406456	total: 1.01s	remaining: 5.79ms
526:	learn: 0.5404662	total: 1.02s	remaining: 3.86ms
527:	learn: 0.5403396	total: 1.02s	remaining: 1.93ms
528:	learn: 0.5401720	total: 1.02s	remaining: 0us
0:	learn: 0.6601681	total: 6.73ms	remaining: 2.02s
1:	learn: 0.6340036	total: 13.4ms	remaining: 2.01s
2:	learn: 0.6085283	total: 19.6ms	remaining: 1.96s
3:	learn: 0.5888337	total: 24.9ms	remaining: 1.85s
4:	learn: 0.5690462	total: 30.1ms	remaining: 1.79s
5:	learn: 0.5544280	total: 35ms	remaining: 1.72s
6:	learn: 0.5406392	total: 40ms	remaining: 1.69s
7:	learn: 0.5315423	total: 45ms	remaining: 1.65s
8:	learn: 0.5168823	total: 49.9ms	remaining: 1.62s
9:	learn: 0.5079255	total: 55ms	remaining: 1.6s
10:	learn: 0.4945778	total: 

C:\Users\vitya\AppData\Local\Temp\ipykernel_5568\3372871564.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),


...

0:	learn: 0.6487266	total: 2.71ms	remaining: 1.89s
1:	learn: 0.6116739	total: 5.43ms	remaining: 1.9s
2:	learn: 0.5864168	total: 7.71ms	remaining: 1.79s
3:	learn: 0.5607218	total: 10ms	remaining: 1.74s
4:	learn: 0.5387802	total: 12.3ms	remaining: 1.7s
5:	learn: 0.5228933	total: 14.6ms	remaining: 1.69s
6:	learn: 0.5091220	total: 16.8ms	remaining: 1.66s
7:	learn: 0.4953730	total: 19.1ms	remaining: 1.65s
8:	learn: 0.4866430	total: 21.4ms	remaining: 1.64s
9:	learn: 0.4795509	total: 24ms	remaining: 1.65s
10:	learn: 0.4713981	total: 26.4ms	remaining: 1.65s
11:	learn: 0.4649603	total: 28.9ms	remaining: 1.66s
12:	learn: 0.4583891	total: 31.2ms	remaining: 1.65s
13:	learn: 0.4532013	total: 33.4ms	remaining: 1.64s
14:	learn: 0.4493993	total: 35.6ms	remaining: 1.63s
15:	learn: 0.4458818	total: 37.7ms	remaining: 1.61s
16:	learn: 0.4412924	total: 40.2ms	remaining: 1.61s
17:	learn: 0.4381995	total: 42.5ms	remaining: 1.61s
18:	learn: 0.4338352	total: 44.9ms	remaining: 1.61s
19:	learn: 0.4311347	total: 

[I 2024-02-17 09:37:32,483] Trial 48 finished with value: 0.8010771169932394 and parameters: {'iterations': 700, 'learning_rate': 0.09957448940222743, 'depth': 5}. Best is trial 22 with value: 0.807264810358657.
C:\Users\vitya\AppData\Local\Temp\ipykernel_5568\3372871564.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),


0:	learn: 0.6830584	total: 2.19ms	remaining: 846ms
1:	learn: 0.6725277	total: 4.1ms	remaining: 792ms
2:	learn: 0.6645925	total: 6.16ms	remaining: 791ms
3:	learn: 0.6570587	total: 8.11ms	remaining: 778ms
4:	learn: 0.6495191	total: 10.2ms	remaining: 778ms
5:	learn: 0.6407220	total: 12.2ms	remaining: 775ms
6:	learn: 0.6334329	total: 14.4ms	remaining: 786ms
7:	learn: 0.6274929	total: 16.4ms	remaining: 781ms
8:	learn: 0.6207248	total: 18.6ms	remaining: 784ms
9:	learn: 0.6137044	total: 20.9ms	remaining: 790ms
10:	learn: 0.6078610	total: 22.8ms	remaining: 781ms
11:	learn: 0.6020072	total: 25.1ms	remaining: 785ms
12:	learn: 0.5952612	total: 27.1ms	remaining: 782ms
13:	learn: 0.5906681	total: 29.2ms	remaining: 781ms
14:	learn: 0.5860152	total: 31.2ms	remaining: 775ms
15:	learn: 0.5816381	total: 33.2ms	remaining: 773ms
16:	learn: 0.5770333	total: 35ms	remaining: 764ms
17:	learn: 0.5724069	total: 37ms	remaining: 761ms
18:	learn: 0.5675227	total: 39.1ms	remaining: 759ms
19:	learn: 0.5638269	total:

[I 2024-02-17 09:37:35,074] Trial 49 finished with value: 0.8001604216798442 and parameters: {'iterations': 388, 'learning_rate': 0.023799908978139814, 'depth': 4}. Best is trial 22 with value: 0.807264810358657.


347:	learn: 0.3928362	total: 715ms	remaining: 82.1ms
348:	learn: 0.3926788	total: 716ms	remaining: 80.1ms
349:	learn: 0.3925084	total: 718ms	remaining: 78ms
350:	learn: 0.3923873	total: 720ms	remaining: 75.9ms
351:	learn: 0.3922895	total: 722ms	remaining: 73.9ms
352:	learn: 0.3920969	total: 724ms	remaining: 71.8ms
353:	learn: 0.3919277	total: 726ms	remaining: 69.7ms
354:	learn: 0.3918469	total: 728ms	remaining: 67.7ms
355:	learn: 0.3915476	total: 730ms	remaining: 65.6ms
356:	learn: 0.3914610	total: 732ms	remaining: 63.6ms
357:	learn: 0.3913913	total: 735ms	remaining: 61.6ms
358:	learn: 0.3913099	total: 737ms	remaining: 59.5ms
359:	learn: 0.3912342	total: 739ms	remaining: 57.5ms
360:	learn: 0.3911691	total: 741ms	remaining: 55.5ms
361:	learn: 0.3911006	total: 743ms	remaining: 53.4ms
362:	learn: 0.3910032	total: 746ms	remaining: 51.4ms
363:	learn: 0.3909075	total: 748ms	remaining: 49.3ms
364:	learn: 0.3907070	total: 750ms	remaining: 47.3ms
365:	learn: 0.3905195	total: 752ms	remaining: 45